In [ ]:
import sys
!{sys.executable} -m pip install python-docx
!{sys.executable} -m pip install --upgrade transformers[torch] datasets evaluate sacremoses
!{sys.executable} -m pip install --upgrade mlxtend wandb
!{sys.executable} -m pip install --upgrade peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.9 MB/s eta 0:00:00
  Attempting uninst

In [ ]:
import docx
import os, re
from collections import Counter

# Data loading code

Counter(labels)


Counter({0: 168, 1: 195})

In [ ]:

from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(labels, labels)
dummy_clf.score(labels, labels)

0.5371900826446281

In [ ]:
!nvidia-smi

Fri Nov 24 21:58:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Print details about the model

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [ ]:
print(model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1280)
    (wpe): Embedding(1536, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear(
            in_features=1280, out_features=3840, bias=True
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1280, out_features=4, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=4, out_features=3840, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0

P-tuning, settings as in here: https://huggingface.co/docs/peft/task_guides/ptuning-seq-classification

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import numpy as np
import wandb, gc, os, pickle, datasets, torch, evaluate

import transformers
import accelerate
import peft
from peft import LoraConfig, PromptEncoderConfig, get_peft_model

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from datasets import concatenate_datasets, Dataset
from transformers import (
    GPT2Config,
    AutoTokenizer,
    GPT2Model,
    GPT2TokenizerFast,
    GPT2ForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments)

transformers.logging.set_verbosity_error()

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def preprocess_function(examples, tokenizer):
    return tokenizer.batch_encode_plus(examples["text"], is_split_into_words=False, truncation=True, max_length=None)


model_name = "sdadas/polish-gpt2-large"
runtime_name = "gpt2large_ptuning"

tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id # or .eos_token ?

dataset_dict = {'label': labels, 'text': texts}
dataset_hf = Dataset.from_dict(dataset_dict)
dataset_hf = dataset_hf.map(lambda p: preprocess_function(p, tokenizer), batched=True)

labels = np.array(labels)
texts = np.array(texts)

true_labels = [] # ground truth
predictions = [] # model predictions

batch_size = 2
fold_nr=0
all_results = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

kf = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kf.split(texts):

  print(f'Fold_nr= {fold_nr}') ; fold_nr += 1

  model_config = GPT2Config.from_pretrained(model_name, num_labels=2, pad_token_id = tokenizer.eos_token_id)
  model = GPT2ForSequenceClassification.from_pretrained(model_name, config=model_config)

  print_trainable_parameters(model)

  config = PromptEncoderConfig(task_type="SEQ_CLS", num_virtual_tokens=20, encoder_hidden_size=128)


  lora_model = get_peft_model(model, config)
  print_trainable_parameters(lora_model)

  # https://colab.research.google.com/github/gmihaila/ml_things/blob/master/notebooks/pytorch/gpt2_finetune_classification.ipynb
  model.resize_token_embeddings(len(tokenizer))
  model.config.pad_token_id = model.config.eos_token_id
  model.to(device)

  training_args = TrainingArguments(
      do_eval=True,
      output_dir='./results-'+runtime_name,          # output directory
      overwrite_output_dir=True,
      num_train_epochs=5,              # total number of training epochs
      per_device_train_batch_size=batch_size,   # 8 @A100 batch size per device during training
      per_device_eval_batch_size=batch_size,    # 8 @A100 batch size for evaluation
      warmup_steps=100,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=80,
      evaluation_strategy="steps",     # Evaluation is done at the end of each epoch, alternatives: "steps", "epoch"
      save_strategy="steps",           # Save is done at the end of each epoch, alternatives: "steps", "epoch"
      # save_steps=500                 # Number of updates steps before two checkpoint saves if save_strategy="steps",
      learning_rate=1e-4,
      eval_steps = 25,
      save_total_limit = 1,
      load_best_model_at_end=True,     # Whether or not to load the best model found during training at the end of training.
      metric_for_best_model='accuracy',
      disable_tqdm=False,
      run_name = runtime_name
  )

  train_dataset = dataset_hf.select(train_index)
  test_dataset = dataset_hf.select(test_index)

  # let's have validation set from 5% of train:
  train_dataset = train_dataset.train_test_split(test_size=0.05, shuffle=True)

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='longest', return_tensors='pt')

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset['train'],  # train part of train
      eval_dataset=train_dataset['test'],  # this is validation
      data_collator=data_collator,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )
  trainer.train()

  predictions_obj = trainer.predict(test_dataset)
  y_pred = np.argmax(predictions_obj.predictions, axis=1)
  test_dataset = test_dataset.add_column("y_pred", y_pred)

  predictions.append( y_pred )
  true_labels.append( labels[test_index] )


print( classification_report(np.concatenate(true_labels).ravel().tolist(), np.concatenate(predictions).ravel().tolist() ) )


Map:   0%|          | 0/363 [00:00<?, ? examples/s]

Fold_nr= 0
trainable params: 775895040 || all params: 775895040 || trainable%: 100.00
trainable params: 376320 || all params: 776268800 || trainable%: 0.05


/usr/local/lib/python3.10/dist-packages/peft/tuners/p_tuning.py:146: UserWarning: for MLP, the `encoder_num_layers` is ignored. Exactly 2 MLP layers are used.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
25,No log,0.737078,0.588235,0.222222,0.333333,0.166667
50,No log,0.712205,0.647059,0.400000,0.500000,0.333333
75,No log,0.719619,0.588235,0.533333,0.444444,0.666667
100,0.712300,0.703144,0.588235,0.533333,0.444444,0.666667
125,0.712300,0.727873,0.588235,0.588235,0.454545,0.833333
150,0.712300,0.676152,0.588235,0.533333,0.444444,0.666667
175,0.659700,0.696995,0.647059,0.625000,0.500000,0.833333
200,0.659700,0.701667,0.588235,0.588235,0.454545,0.833333
225,0.659700,0.678464,0.647059,0.625000,0.500000,0.833333
250,0.695800,0.656086,0.529412,0.428571,0.375000,0.500000


Flattening the indices:   0%|          | 0/37 [00:00<?, ? examples/s]

...

Fold_nr= 9
trainable params: 775895040 || all params: 775895040 || trainable%: 100.00
trainable params: 376320 || all params: 776268800 || trainable%: 0.05


/usr/local/lib/python3.10/dist-packages/peft/tuners/p_tuning.py:146: UserWarning: for MLP, the `encoder_num_layers` is ignored. Exactly 2 MLP layers are used.
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
25,No log,0.792420,0.705882,0.444444,1.000000,0.285714
50,No log,0.772268,0.529412,0.333333,0.400000,0.285714
75,No log,0.731099,0.470588,0.307692,0.333333,0.285714
100,0.810100,0.738240,0.411765,0.444444,0.363636,0.571429
125,0.810100,0.801765,0.529412,0.555556,0.454545,0.714286
150,0.810100,0.752600,0.529412,0.500000,0.444444,0.571429
175,0.659000,0.748840,0.529412,0.500000,0.444444,0.571429
200,0.659000,0.793076,0.588235,0.588235,0.500000,0.714286
225,0.659000,0.780053,0.529412,0.500000,0.444444,0.571429
250,0.699200,0.762030,0.529412,0.500000,0.444444,0.571429


Flattening the indices:   0%|          | 0/36 [00:00<?, ? examples/s]

              precision    recall  f1-score   support

           0       0.65      0.51      0.57       168
           1       0.64      0.76      0.70       195

    accuracy                           0.64       363
   macro avg       0.65      0.64      0.63       363
weighted avg       0.64      0.64      0.64       363



LORA according to config in the paper:

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import numpy as np
import wandb, gc, os, pickle, datasets, torch, evaluate

import transformers
import accelerate
import peft
from peft import LoraConfig, PromptEncoderConfig, get_peft_model

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from datasets import concatenate_datasets, Dataset
from transformers import (
    GPT2Config,
    AutoTokenizer,
    GPT2Model,
    GPT2TokenizerFast,
    GPT2ForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments)

transformers.logging.set_verbosity_error()

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def preprocess_function(examples, tokenizer):
    return tokenizer.batch_encode_plus(examples["text"], is_split_into_words=False, truncation=True, max_length=None)


model_name = "sdadas/polish-gpt2-large"
runtime_name = "gpt2large_lora"

tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id # or .eos_token ?

dataset_dict = {'label': labels, 'text': texts}
dataset_hf = Dataset.from_dict(dataset_dict)
dataset_hf = dataset_hf.map(lambda p: preprocess_function(p, tokenizer), batched=True)

labels = np.array(labels)
texts = np.array(texts)

true_labels = [] # ground truth
predictions = [] # model predictions

batch_size = 1
fold_nr=0
all_results = []
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

kf = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kf.split(texts):

  print(f'Fold_nr= {fold_nr}') ; fold_nr += 1

  model_config = GPT2Config.from_pretrained(model_name, num_labels=2, pad_token_id = tokenizer.eos_token_id)
  model = GPT2ForSequenceClassification.from_pretrained(model_name, config=model_config)

  print_trainable_parameters(model)

  config = LoraConfig(
      r=4,
      lora_alpha=32,
      target_modules=["c_attn"],
      lora_dropout=0.1,
      bias="lora_only",
      modules_to_save=["classifier"],
  )

  lora_model = get_peft_model(model, config)
  print_trainable_parameters(lora_model)

  model.resize_token_embeddings(len(tokenizer))
  model.config.pad_token_id = model.config.eos_token_id
  model.to(device)

  training_args = TrainingArguments(
      do_eval=True,
      output_dir='./results-'+runtime_name,          # output directory
      overwrite_output_dir=True,
      num_train_epochs=5,              # total number of training epochs
      per_device_train_batch_size=batch_size,   # 8 @A100 batch size per device during training
      per_device_eval_batch_size=batch_size,    # 8 @A100 batch size for evaluation
      warmup_steps=100,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=80,
      evaluation_strategy="steps",     # Evaluation is done at the end of each epoch, alternatives: "steps", "epoch"
      save_strategy="steps",           # Save is done at the end of each epoch, alternatives: "steps", "epoch"
      # save_steps=500                 # Number of updates steps before two checkpoint saves if save_strategy="steps",
      learning_rate=2e-4,
      eval_steps = 25,
      save_total_limit = 1,
      load_best_model_at_end=True,     # Whether or not to load the best model found during training at the end of training.
      metric_for_best_model='accuracy',
      disable_tqdm=False,
      run_name = runtime_name
  )

  train_dataset = dataset_hf.select(train_index)
  test_dataset = dataset_hf.select(test_index)

  # let's have validation set from 5% of train:
  train_dataset = train_dataset.train_test_split(test_size=0.05, shuffle=True)

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='longest', return_tensors='pt')

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset['train'],  # train part of train
      eval_dataset=train_dataset['test'],  # this is validation
      data_collator=data_collator,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )
  trainer.train()

  predictions_obj = trainer.predict(test_dataset)
  y_pred = np.argmax(predictions_obj.predictions, axis=1)
  test_dataset = test_dataset.add_column("y_pred", y_pred)

  predictions.append( y_pred )
  true_labels.append( labels[test_index] )


print( classification_report(np.concatenate(true_labels).ravel().tolist(), np.concatenate(predictions).ravel().tolist() ) )


Map:   0%|          | 0/363 [00:00<?, ? examples/s]

Fold_nr= 0
trainable params: 775895040 || all params: 775895040 || trainable%: 100.00


/usr/local/lib/python3.10/dist-packages/peft/tuners/lora.py:240: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 875520 || all params: 776632320 || trainable%: 0.11


wandb: Currently logged in as: a-wawer. Use `wandb login --relogin` to force relogin


...

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora.py:240: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 875520 || all params: 776632320 || trainable%: 0.11


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
25,No log,0.727659,0.411765,0.285714,0.285714,0.285714
50,No log,0.687152,0.470588,0.400000,0.375000,0.428571
75,No log,0.669734,0.647059,0.250000,1.000000,0.142857
100,0.797100,0.530645,0.764706,0.600000,1.000000,0.428571
125,0.797100,0.387007,0.764706,0.600000,1.000000,0.428571
150,0.797100,0.307602,0.705882,0.444444,1.000000,0.285714
175,0.715600,0.557158,0.764706,0.777778,0.636364,1.000000
200,0.715600,2.118684,0.411765,0.583333,0.411765,1.000000
225,0.715600,2.874540,0.411765,0.583333,0.411765,1.000000
250,1.643000,1.278973,0.764706,0.600000,1.000000,0.428571


Flattening the indices:   0%|          | 0/36 [00:00<?, ? examples/s]

              precision    recall  f1-score   support

           0       0.89      0.87      0.88       168
           1       0.89      0.91      0.90       195

    accuracy                           0.89       363
   macro avg       0.89      0.89      0.89       363
weighted avg       0.89      0.89      0.89       363

